# Distances

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (14:34:46) Alles bereit (+0.0s)


## Words

In [ ]:
print('ok?')

with get_veclib('virtues',['wvecs'],autocommit=False) as vl:
    dfdist=pd.DataFrame(dict(vl.items())).T
print('done')
dfdist

In [2]:

def do_cdist(objd):
    res=cdist(**objd)
    if res is None: res=pd.DataFrame()
    return res

def cdist(word,period=None,run=None,prefix='cdist',neighbors=None,max_num=10000,num_runs=10,num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
    index_cols=['word','neighbor','period','run']
    argd=dict(
        word=word,period=period,run=run,prefix=prefix,
        neighbors=neighbors,max_num=max_num,num_runs=num_runs,
        num_proc=num_proc,force=force,progress=progress,
        cache_only=cache_only,cache=cache,vl=vl
    )
    odf=None
    
    if type(word)!=str:
        objs=[{**argd, **{'word':w, 'progress':False, 'num_proc':1}} for w in word]
        odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across words', progress=progress, use_threads=False))
        return odf if not cache_only else pd.DataFrame()
    
    with get_veclib(prefix=word,folders=['wvecs'],autocommit=False) as vl:
        if period is None:
            objs=[{**argd, **{'period':prd, 'progress':False, 'num_proc':1, 'vl':vl}} for prd in get_default_periods()]
            odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress, use_threads=True))
            return odf if not cache_only else pd.DataFrame()
        if run is None:
            objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1, 'vl':vl}} for run in range(num_runs)]
            odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress, use_threads=True))
            return odf if not cache_only else pd.DataFrame()

        # load vecs?
        qstr=f'{period}_{run:02}'
        sdist=vl.get(qstr)
        if sdist is None: return pd.DataFrame()
#         sdist=sdist[set(sdist.index)-{word}]
#         sdist=sdist*-1 if sdist.min()<-0.1 else sdist
        if neighbors: sdist=sdist[(set(neighbors) & set(sdist.index))]
        wddf=pd.DataFrame()
        wddf['cdist']=sdist
        wddf=wddf.rename_axis('neighbor').sort_values('cdist')
        wddf['word']=word
        wddf['period']=period
        wddf['run']=run
        wddf=wddf.reset_index().set_index(index_cols)
        return wddf


In [3]:
# dfvecs=vecs('1780-1785',1)
words=get_valid_words()
# words[:100]

In [4]:
w=random.choice(words)
w

'din'

In [5]:
# %%timeit
# cdist(w,num_proc=8)

In [6]:
# %%timeit
# cdist(w,num_proc=1)

In [7]:
# %%timeit
# cdist(w,num_proc=4)

In [10]:
swords=random.sample(words[:1000],4)
swords

['fear', 'sale', 'selection', 'book']

In [11]:
# %%timeit
# cdist(swords,num_proc=1)

In [12]:
# %%timeit
# cdist(swords,num_proc=4)

In [15]:
# cdist(random.sample(words,100),num_proc=4)

[Koselleck] (14:37:31) ok? (+8.9s)
[Koselleck] (14:37:34) done (+2.8s)


,a',a's,aa,aaa,aad,aal,aam,aaron,aaronson,aas,...,zoom,zoroaster,zos,zr,zs,zu,zulu,zulus,zurich,zz
1780-1785_01,2.274839,2.522329,2.417590,1.885422,2.233348,2.026744,NaN,NaN,NaN,1.813828,...,NaN,NaN,NaN,2.102728,2.167725,NaN,NaN,NaN,NaN,NaN
1720-1725_01,1.993256,1.855927,2.178085,1.998352,1.710748,1.949194,1.979302,2.017306,NaN,1.701567,...,NaN,NaN,NaN,2.111788,2.056255,2.037828,NaN,NaN,NaN,NaN
1720-1725_02,1.885313,1.896931,2.204282,1.899887,1.891192,1.959113,NaN,1.922307,NaN,1.808390,...,NaN,NaN,NaN,2.093445,2.228867,1.833843,NaN,NaN,1.950428,NaN
1720-1725_03,2.009534,1.939957,2.171373,2.069225,1.986268,1.966106,NaN,NaN,NaN,1.785482,...,NaN,NaN,NaN,2.142313,2.280167,2.038747,NaN,NaN,1.850128,NaN
1720-1725_04,2.044952,1.870314,2.304555,1.715736,1.639848,1.649849,2.051018,1.802204,NaN,1.709249,...,NaN,NaN,NaN,2.177713,2.159464,1.909513,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955-1960_06,2.284276,NaN,1.886938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.197970,NaN
1955-1960_07,1.985536,NaN,1.576267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.289087,NaN
1955-1960_08,2.068463,NaN,1.713967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.059180,NaN
1955-1960_09,2.172709,NaN,1.734996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.485581,NaN


In [14]:
dfdist=cdist('virtues',num_proc=4)
dfdist

Measuring cosine distances across periods [x4]: 100%|██████████| 48/48 [00:11<00:00,  4.20it/s]


cdist
word    neighbor   period    run              
virtues virtues    1720-1725 1    2.220446e-16
        generosity 1720-1725 1    8.991674e-01
        piety      1720-1725 1    9.270416e-01
        wisdom     1720-1725 1    9.385054e-01
        welfare    1720-1725 1    9.614458e-01
...                                        ...
        stamford   1955-1960 10   2.779507e+00
        flew       1955-1960 10   2.796643e+00
        ashley     1955-1960 10   2.798385e+00
        paddington 1955-1960 10   2.808796e+00
        waterloo   1955-1960 10   2.868152e+00

[4800000 rows x 1 columns]

In [22]:
to_nbr('virtues','1720-1725')

Measuring cosine distances across runs [x1]: 100%|██████████| 10/10 [00:00<00:00, 56.18it/s]


count  num_runs         cdist  rank
word    neighbor    period                                        
virtues virtues     1720-1725     10        10 -1.554312e-16     1
        wisdom      1720-1725     10        10  9.237895e-01     2
        qualities   1720-1725     10        10  9.351535e-01     3
        esteem      1720-1725     10        10  9.590707e-01     4
        piety       1720-1725     10        10  9.694210e-01     5
...                              ...       ...           ...   ...
        pride       1720-1725      3         5  1.079828e+00    96
        supposing   1720-1725      3         9  1.091652e+00    97
        obligations 1720-1725      3        10  1.101753e+00    98
        malicious   1720-1725      3         8  1.102253e+00    99
        truths      1720-1725      3         5  1.104915e+00   100

[100 rows x 4 columns]

In [13]:
stop

NameError: name 'stop' is not defined

In [ ]:
# cdist(random.sample(words,100),num_proc=4)

In [ ]:
cdist('virtues')

In [ ]:
# # %%timeit
# for word in tqdm(words[:1000]):
#     cdist(word,'1760-1765',progress=False)

In [ ]:
# %%timeit
# cdist('culture','1760-1765',1)

In [ ]:
# dists=cdist('culture',neighbors=['plant','soil','education'],num_proc=1)
# dists

In [ ]:
# distvecs('1770-1775',1)